In [1]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os
from PIL import Image
import numpy as np

In [6]:
window = tk.Tk()
window.title("Face Recognition system")

l1 = tk.Label(window, text="Name", font=("Algerian",20))
l1.grid(column=0, row=0)
t1 = tk.Entry(window, width=50, bd=5)
t1.grid(column=1, row=0)

l2 = tk.Label(window, text="Age", font=("Algerian",20))
l2.grid(column=0, row=1)
t2 = tk.Entry(window, width=50, bd=5)
t2.grid(column=1, row=1)

l3 = tk.Label(window, text="Address", font=("Algerian",20))
l3.grid(column=0, row=2)
t3 = tk.Entry(window, width=50, bd=5)
t3.grid(column=1, row=2)


b1 = tk.Button(window, text="Training", font=("Algerian",20),bg="orange",fg="red")
b1.grid(column=0, row=4)

b2 = tk.Button(window, text="Detect the face", font=("Algerian",20), bg="green", fg="orange")
b2.grid(column=1, row=4)

b3=tk.Button(window, text="Generate dataset", font=("Algerian",20), bg="pink", fg="black")
b3.grid(column=2, row=4)

window.geometry("800x200")
window.mainloop()

# connecting to database

In [2]:
import cv2
import os
from PIL import Image
import numpy as np

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import tkinter as tk
from tkinter import messagebox

# ---------------- GUI Setup -----------------
window = tk.Tk()
window.title("Face Recognition system")

l1 = tk.Label(window, text="Name", font=("Algerian",20))
l1.grid(column=0, row=0)
t1 = tk.Entry(window, width=50, bd=5)
t1.grid(column=1, row=0)

l2 = tk.Label(window, text="Age", font=("Algerian",20))
l2.grid(column=0, row=1)
t2 = tk.Entry(window, width=50, bd=5)
t2.grid(column=1, row=1)

l3 = tk.Label(window, text="Address", font=("Algerian",20))
l3.grid(column=0, row=2)
t3 = tk.Entry(window, width=50, bd=5)
t3.grid(column=1, row=2)

# ---------------- TRAIN CLASSIFIER -----------------
def train_classifier():
    data_dir = "C:/Users/anany/OneDrive/Desktop/data"   # ✅ fixed path
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')   # grayscale
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])  # filename format: name.id.sample.jpg

        faces.append(imageNp)
        ids.append(id)

    ids = np.array(ids)

    # Train classifier and save
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    messagebox.showinfo("Result","Model training completed!")

b1 = tk.Button(window, text="Training", font=("Algerian",20),
               bg="orange", fg="red", command=train_classifier)
b1.grid(column=0, row=4)

# ---------------- FACE DETECTION -----------------
def detect_face():
    cascPath = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    faceCascade = cv2.CascadeClassifier(cascPath)
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)
        coords = []

        for (x,y,w,h) in features:
            cv2.rectangle(img, (x,y), (x+w,y+h), color, 2 )
            id, pred = clf.predict(gray_image[y:y+h, x:x+w])
            confidence = int(100 * (1 - pred/300))

            if confidence > 75:
                if id == 1:
                    cv2.putText(img, "Ananya", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 
                                0.8, color, 1, cv2.LINE_AA)
                elif id == 2:
                    cv2.putText(img, "Manish", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 
                                0.8, color, 1, cv2.LINE_AA)
            else:
                cv2.putText(img, "UNKNOWN", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 
                            0.8, (0,0,255), 1, cv2.LINE_AA)

            coords = [x,y,w,h]
        return coords

    def recognize(img, clf, faceCascade):
        draw_boundary(img, faceCascade, 1.1, 10, (255,255,255), "Face", clf)
        return img

    video_capture = cv2.VideoCapture(0)

    while True:
        ret, img = video_capture.read()
        img = recognize(img, clf, faceCascade)
        cv2.imshow("Face Detection", img)

        if cv2.waitKey(1) == 13:   # Enter key to stop
            break

    video_capture.release()
    cv2.destroyAllWindows()

# ✅ connect button with detection function
b2 = tk.Button(window, text="Detect the face", font=("Algerian",20),
               bg="green", fg="orange", command=detect_face)
b2.grid(column=1, row=4)

# ---------------- DATASET GENERATION (OPTIONAL) -----------------
def generate_dataset():
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if faces is ():
            return None
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            return cropped_face

    cap = cv2.VideoCapture(0)
    id = 1  # you can set this dynamically
    img_id = 0

    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id += 1
            face = cv2.resize(face_cropped(frame), (200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            file_name_path = f"data/user.{id}.{img_id}.jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow("Cropped Face", face)

        if cv2.waitKey(1) == 13 or int(img_id) == 50:  # Enter or 50 images
            break

    cap.release()
    cv2.destroyAllWindows()
    messagebox.showinfo("Result","Dataset generation completed!")

b3 = tk.Button(window, text="Generate dataset", font=("Algerian",20),
               bg="pink", fg="black", command=generate_dataset)
b3.grid(column=2, row=4)

# ---------------- MAIN LOOP -----------------
window.geometry("800x300")
window.mainloop()

        
     


<>:114: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
<>:114: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
C:\Users\anany\AppData\Local\Temp\ipykernel_6656\2693994856.py:114: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
  if faces is ():


In [ ]:
window = tk.Tk()
window.title("Face Recognition system")

l1 = tk.Label(window, text="Name", font=("Algerian",20))
l1.grid(column=0, row=0)
t1 = tk.Entry(window, width=50, bd=5)
t1.grid(column=1, row=0)

l2 = tk.Label(window, text="Age", font=("Algerian",20))
l2.grid(column=0, row=1)
t2 = tk.Entry(window, width=50, bd=5)
t2.grid(column=1, row=1)

l3 = tk.Label(window, text="Address", font=("Algerian",20))
l3.grid(column=0, row=2)
t3 = tk.Entry(window, width=50, bd=5)
t3.grid(column=1, row=2)

def train_classifier():
    data_dir = "C:Users/anany/OneDrive/Desktop/data"
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')   # convert to grayscale
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])

        faces.append(imageNp)
        ids.append(id)

    ids = np.array(ids)

    # Train the classifier and save
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    messagebox.showinfo("Result","Model trained completed.....")
    
b1 = tk.Button(window, text="Training", font=("Algerian",20),bg="orange",fg="red", command=train_classifier)
b1.grid(column=0, row=4)

def detect_face():
        def draw_boundary(image, classifier, scaleFactor, minNeighbors, color, text, clf):
         gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)
    
        
        
        for (x,y,w,h) in features:
            cv2.rectangle(img, (x,y), (x+w,y+h), color, 2 )
            
            id, pred = clf.predict(gray_image[y:y+h,x:x+w])
            confidence = int(100*(1-pred/300))
            
            if confidence>75:
                if id==1:
                    cv2.putText(img, "Ananya", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
                if id==2:
                    cv2.putText(img, "Manish", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            else:
                cv2.putText(img, "UNKNOWN", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)
    
            coords = [x,y,w,h]
        return coords
        
        def recognize(image,clf,faceCascade):
          coords = draw_boundary(image,faceCascade,1.1,10,(255,255,255),"Face",clf)
        return img

# loading classifier
#faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

cascPath = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

video_capture = cv2.VideoCapture(0)

while True:
    ret, img = video_capture.read()
    img= recognize(img,clf,faceCascade)
    cv2.imshow("face Detection", img)
    
    if cv2.waitKey(1)==13:
           break
video_capture.release()
cv2.destroyAllWindows()

b2 = tk.Button(window, text="Detect the face", font=("Algerian",20), bg="green", fg="orange")
b2.grid(column=1, row=4)

b3=tk.Button(window, text="Generate dataset", font=("Algerian",20), bg="pink", fg="black")
b3.grid(column=2, row=4)

window.geometry("800x300")
window.mainloop()
